In [1]:
import pandas as pd
import json

file_path_nechi = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/Caudal_Medio_Dia_Historico_Nechi.csv'
file_path_san_jacinto = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/Caudal_Medio_Dia_Historico_San_Jacinto_del_Cauca.csv'

data_nechi = pd.read_csv(file_path_nechi, sep=';', encoding='ISO-8859-1')
data_san_jacinto = pd.read_csv(file_path_san_jacinto, sep=';', encoding='ISO-8859-1')


In [3]:
def transform_data(data, lat, lon):
    melted_data = data.melt(id_vars=["year", "month"], var_name="day", value_name="Caudal")
    melted_data.dropna(subset=["Caudal"], inplace=True)
    melted_data['day'] = melted_data['day'].astype(int)
    melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']])
    melted_data.sort_values('Fecha', inplace=True)
    melted_data['Fecha'] = melted_data['Fecha'].astype(str)
    melted_data['Latitud'] = lat
    melted_data['Longitud'] = lon
    return melted_data

data_nechi_transformed = transform_data(data_nechi, 8.030111111, -74.78555556)
data_san_jacinto_transformed = transform_data(data_san_jacinto, 8.344555556, -74.56122222)


In [4]:
combined_data = pd.concat([data_nechi_transformed, data_san_jacinto_transformed], ignore_index=True)

In [5]:
def df_to_geojson(df, properties, lat='Latitud', lon='Longitud'):
    geojson = {'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Point',
                'coordinates': [row[lon], row[lat]]
            }
        }
        for prop in properties:
            if isinstance(row[prop], pd.Timestamp):
                feature['properties'][prop] = row[prop].isoformat()
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [6]:
properties = ['Fecha', 'Caudal']
geojson_data = df_to_geojson(combined_data, properties)

geojson_path = 'C:/Users/Mateo Grisales/Documents/NUVU/Caudales.geojson'
with open(geojson_path, 'w') as f:
    json.dump(geojson_data, f)

print(f"GeoJSON file created at {geojson_path}")


GeoJSON file created at C:/Users/Mateo Grisales/Downloads/Caudal_Nechi.geojson
